# validate results

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import argparse
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import default_rng

from activitysim.cli import run
from activitysim.core import inject
from activitysim.core import tracing
from activitysim.core import config
from activitysim.core import pipeline
from activitysim.core import chunk
from activitysim.core import simulate
from activitysim.core import logit
from activitysim.abm.models.util import estimation
from activitysim.core import expressions
from activitysim.core.util import assign_in_place

In [4]:
pd.set_option("max_columns", 500)

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_mtc")

In [6]:
os.chdir(example_dir)

In [7]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
# args = parser.parse_args()
# parser.parse_args(['--sum', '7', '-1', '42'])
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data'])
#run.run(args)  # 2mins full example run
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [8]:
from fru_utils import run_trip_mode_choice
trips_df_ru = run_trip_mode_choice()
trips_df = run_trip_mode_choice(choose_individual_max_utility=False)

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)
Done


register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'atwork' (27 trips)
trip_mode_choice tour_type 'eatout' (33 trips)
trip_mode_choice tour_type 'escort' (6 trips)
trip_mode_choice tour_type 'othdiscr' (43 trips)
trip_mode_choice tour_type 'othmaint' (46 trips)
trip_mode_choice tour_type 'school' (37 trips)
trip_mode_choice tour_type 'shopping' (77 trips)
trip_mode_choice tour_type 'social' (19 trips)
trip_mode_choice tour_type 'univ' (26 trips)
trip_mode_choice tour_type 'work' (168 trips)
Done


In [9]:
c_ = ["trip_mode", "mode_choice_logsum"]
trips = trips_df_ru[c_].merge(trips_df[c_], left_index=True, right_index=True, how="outer", suffixes=["_fru", "_asim"])
pd.merge(trips_df_ru.trip_mode.value_counts(), trips_df.trip_mode.value_counts(), left_index=True, right_index=True,
         suffixes=["_fru", "_asim"])

,trip_mode_fru,trip_mode_asim
WALK,299,304
WALK_LOC,92,85
WALK_LRF,61,53
BIKE,17,17
DRIVEALONEFREE,6,4
SHARED2FREE,3,4
TNC_SINGLE,2,4
WALK_HVY,2,4


In [62]:
if np.allclose(trips.mode_choice_logsum_fru, trips.mode_choice_logsum_asim):
    print("Logsums agree")
else:
    ((trips.mode_choice_logsum_fru - trips.mode_choice_logsum_asim) / trips.mode_choice_logsum_fru).hist(bins=50)

Logsums agree


# Run MC validation

see logit._each_nest parent_nest_scale for leaf node, and simulate.compute_nested_utilities for discussion of scales
of alternatives

easiest way is to run a simulation

In [14]:
from fru_utils import get_stuff
t, ru, neu, nu, ns, nested_probs, base_probs = get_stuff(do_these_purposes=["social"])

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


Running with %d trips 482
trip_mode_choice tour_type 'social' (19 trips)
Done


In [15]:
nested_probs[0]

,AUTO,NONMOTORIZED,TRANSIT,RIDEHAIL,DRIVEALONE,SHAREDRIDE2,SHAREDRIDE3,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALKACCESS,DRIVEACCESS,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8805121,0.000000e+00,1.000000,0.000000,1.418229e-07,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300854,0.553048,0.146098
8805125,0.000000e+00,1.000000,0.000000,1.333383e-07,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.304288,0.549407,0.146304
8805126,0.000000e+00,1.000000,0.000000,1.666869e-07,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291815,0.562518,0.145667
468119921,0.000000e+00,1.000000,0.000000,5.175690e-08,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.211883,0.005323,0.782795
468119922,0.000000e+00,0.999874,0.000000,1.255768e-04,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.211720,0.005255,0.783024
468119925,0.000000e+00,0.999875,0.000000,1.254853e-04,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.211271,0.005261,0.783468
468119926,0.000000e+00,1.000000,0.000000,5.207466e-08,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.301114,0.004624,0.694262
468119927,0.000000e+00,1.000000,0.000000,7.363066e-08,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.720964,0.001787,0.277249
468119928,0.000000e+00,1.000000,0.000000,2.191280e-08,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108040,0.006816,0.885144


In [16]:
base_probs[0]

,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,
8805121,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.266799e-08,7.843494e-08,2.071998e-08
8805125,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.057327e-08,7.325707e-08,1.950799e-08
8805126,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.864169e-08,9.376443e-08,2.428075e-08
468119921,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.096638e-08,2.754998e-10,4.051502e-08
468119922,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.999874,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.658716e-05,6.599176e-07,9.832967e-05
468119925,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.999875,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.651144e-05,6.601311e-07,9.831370e-05
468119926,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.568041e-08,2.408144e-10,3.615344e-08
468119927,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.308505e-08,1.315547e-10,2.041405e-08
468119928,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.367465e-09,1.493582e-10,1.939597e-08


In [ ]:
# for later: this seems to be a non-driving segment - grab workers or something later?
# for now: implement a run with seeding so I can reproduce these counts

In [453]:
def simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_d, estimator, tr_label, log_alt_losers, trace_column_names):
    trace_label = tracing.extend_trace_label(tr_label, 'eval_nl')
    logit.validate_nest_spec(nest_spec, trace_label)
    raw_utilities = simulate.eval_utilities(spec, trips_segment, locals_d,
                                   log_alt_losers=log_alt_losers,
                                   trace_label=trace_label, have_trace_targets=False,
                                   estimator=estimator, trace_column_names=trace_column_names)
    nested_exp_utilities = simulate.compute_nested_exp_utilities(raw_utilities, nest_spec)
    nested_probabilities = \
        simulate.compute_nested_probabilities(nested_exp_utilities, nest_spec, trace_label=trace_label)
    # global (flattened) leaf probabilities based on relative nest coefficients (in spec order)
    base_probabilities = simulate.compute_base_probabilities(nested_probabilities, nest_spec, spec)    
    return base_probabilities
#simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names)

In [454]:
## caculate probabilities with Asim methodology, should be correct
def gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
          trace_label=None, log_alt_losers=None, trace_column_names=None):
    full_probs = []  # analytical probs

    for primary_purpose, trips_segment in trips_merged.groupby('primary_purpose'):
        #print("trip_mode_choice tour_type '%s' (%s trips)" %
        #            (primary_purpose, len(trips_segment.index), ))
        # name index so tracing knows how to slice
        assert trips_segment.index.name == 'trip_id'

        coefficients = simulate.get_segment_coefficients(model_settings, primary_purpose)

        locals_dict = {}
        locals_dict.update(constants)
        locals_dict.update(coefficients)

        segment_trace_label = tracing.extend_trace_label(trace_label, primary_purpose)

        expressions.annotate_preprocessors(
            trips_segment, locals_dict, skims,
            model_settings, segment_trace_label)

        locals_dict.update(skims)

        spec=simulate.eval_coefficients(model_spec, coefficients, estimator)
        nest_spec = simulate.eval_nest_coefficients(nest_spec, coefficients, segment_trace_label)
        #choices = simple_simulate_rum(trips_segment, spec, nest_spec, locals_dict, estimator, 
        #                              segment_trace_label, log_alt_losers=log_alt_losers, 
        #                              trace_column_names=trace_column_names, seed=seed)
        probs = simple_simulate_probabilities(trips_segment, spec, nest_spec, locals_dict, 
                                              estimator, segment_trace_label, log_alt_losers, 
                                              trace_column_names)
        full_probs.append(probs)
    probs_df = pd.concat(full_probs)
    return probs_df


In [455]:
print(f"{datetime.now()} Start")
pipeline.open_pipeline(resume_after)
inject.get_injectable('preload_injectables', None)
model_name = "trip_mode_choice"
pipeline._PIPELINE.rng().begin_step(model_name)

step_name = model_name
args = {}
checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

trips = inject.get_table('trips')
tours_merged = inject.get_table('tours_merged')
network_los = inject.get_injectable('network_los')
chunk_size = 0

trace_label = 'trip_mode_choice'
model_settings_file_name = 'trip_mode_choice.yaml'
model_settings = config.read_model_settings(model_settings_file_name)

logsum_column_name = model_settings.get('MODE_CHOICE_LOGSUM_COLUMN_NAME')
mode_column_name = 'trip_mode'
trips_df = trips.to_frame()
#print("Running with %d trips", trips_df.shape[0])
tours_merged = tours_merged.to_frame()
tours_merged = tours_merged[model_settings['TOURS_MERGED_CHOOSER_COLUMNS']]
# - trips_merged - merge trips and tours_merged
trips_merged = pd.merge(
    trips_df,
    tours_merged,
    left_on='tour_id',
    right_index=True,
    how="left")
assert trips_merged.index.equals(trips.index)

# setup skim keys
assert ('trip_period' not in trips_merged)
trips_merged['trip_period'] = network_los.skim_time_period_label(trips_merged.depart)

orig_col = 'origin'
dest_col = 'destination'

constants = {}
constants.update(config.get_model_constants(model_settings))
constants.update({
    'ORIGIN': orig_col,
    'DESTINATION': dest_col
})

skim_dict = network_los.get_default_skim_dict()

odt_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=orig_col, dest_key=dest_col,
                                           dim3_key='trip_period')
dot_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=dest_col, dest_key=orig_col,
                                           dim3_key='trip_period')
od_skim_wrapper = skim_dict.wrap('origin', 'destination')

skims = {
    "odt_skims": odt_skim_stack_wrapper,
    "dot_skims": dot_skim_stack_wrapper,
    "od_skims": od_skim_wrapper,
}

model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
nest_spec = config.get_logit_model_settings(model_settings)

estimator = estimation.manager.begin_estimation('trip_mode_choice')


all_choices = []
for i in range(100):
    if i % 10 == 0:
        print(f"{datetime.now()} iteration {i}")
    choices_df = stuff(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
              trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=None)
    all_choices.append(choices_df)
all_choices = pd.concat(all_choices, axis=1)

probs_nl = gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec,
                               estimator, logsum_column_name, mode_column_name,trace_label=trace_label, 
                               log_alt_losers=None, trace_column_names=None)

# update trips table with choices (and potionally logssums)
#trips_df = trips.to_frame()
#
#assign_in_place(trips_df, choices_df)
#assert not trips_df[mode_column_name].isnull().any()


finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

print(f"{datetime.now()} End")

register joint_tour_participants: no rows with household_id in [982875].
estimation bundle trip_mode_choice not in settings file estimation.yaml


2021-09-01 14:59:46.353007 Start
2021-09-01 14:59:52.976736 End


In [463]:
#all_choices.merge(choices_df_asim[['trip_mode']].rename(columns={'trip_mode': 'asim'}), left_index=True, right_index=True)
val_counts = all_choices.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts = val_counts / all_choices.shape[1]
#val_counts = val_counts.merge(choices_df_asim[['trip_mode']].rename(columns={'trip_mode': 'asim'}), left_index=True, right_index=True)
#val_counts['prob_of_asim_choice'] = val_counts.apply(lambda x: x[x.asim], axis=1)  # this is what our simulation says w.r.t. to asim choice
# for 100% and many samples should mostly agree

In [464]:
#val_counts['prob_of_asim_choice'].hist(bins=100);

In [490]:
mode_share_analytical = (probs_nl.sum(axis=0) / probs_nl.shape[0]).to_frame('analytical')
assert np.allclose(mode_share_analytical.sum(), 1)
mode_share_rum = (val_counts.sum(axis=0) / val_counts.shape[0]).to_frame('experiment')
assert np.allclose(mode_share_rum.sum(), 1)
full_share = mode_share_analytical.join(mode_share_rum, how='outer').fillna(0)
with pd.option_context("precision", 3):
    display((100.0 * full_share).T)

,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,DRIVE_COM,DRIVE_EXP,DRIVE_HVY,DRIVE_LOC,DRIVE_LRF,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_COM,WALK_EXP,WALK_HVY,WALK_LOC,WALK_LRF
analytical,3.152,0.852,0.0,0.0,0.0,0.0,0.0,0.0,0.685,0.0,0.129,0.0,0.182,0.260,1.334,63.708,0.0,0.0,0.455,18.355,10.887
experiment,3.243,0.880,0.0,0.0,0.0,0.0,0.0,0.0,0.772,0.0,0.156,0.0,0.174,0.259,1.434,63.243,0.0,0.0,0.394,18.639,10.807


### try zenith normalisation of simple_simulate_rum

In [629]:
# TODO: work out if our formulation and formulation belowi s equivalent.
def compute_nested_utilities_zenith(raw_utilities, nest_spec):
    nested_utilities = pd.DataFrame(index=raw_utilities.index)
    for nest in logit.each_nest(nest_spec, post_order=True):
        name = nest.name
        if nest.is_leaf:
            nested_utilities[name] = \
                raw_utilities[name].astype(float) # / nest.product_of_coefficients  #coefficient
        else:
            with np.errstate(divide='ignore'):
                nested_utilities[name] = \
                    nest.coefficient * np.log(np.exp(nested_utilities[nest.alternatives] / nest.coefficient).sum(axis=1))
                
    # now go over all leaves and correct for scale
    for nest in logit.each_nest(nest_spec, post_order=True):
        name = nest.name
        if nest.is_leaf:
            nested_utilities[name] /= nest.coefficient
    
    return nested_utilities


def simple_simulate_rum_zenith(trips_segment, spec, nest_spec, locals_d, estimator, tr_label, log_alt_losers, trace_column_names, custom_chooser=None, seed=None):
    trace_label = tracing.extend_trace_label(tr_label, 'eval_nl')
    logit.validate_nest_spec(nest_spec, trace_label)
    raw_utilities = simulate.eval_utilities(spec, trips_segment, locals_d,
                                   log_alt_losers=log_alt_losers,
                                   trace_label=trace_label, have_trace_targets=False,
                                   estimator=estimator, trace_column_names=trace_column_names)

    utils_df = compute_nested_utilities_zenith(raw_utilities, nest_spec)

    nests_ = list(logit.each_nest(nest_spec))
    # group nests by level:
    depth = np.max([x.level for x in nests_])
    nest_levels = {x: [] for x in range(1, depth+1)}
    for n in nests_:
        nest_levels[n.level].append(n.name)
    assert len(nest_levels[1]) == 1
    assert nest_levels[1][0] == 'root'
    # make choices
    choices = make_choice(utils_df, nests_, nest_levels, seed)

    return choices

#simple_simulate_rum_zenith(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=1233974)

In [630]:
def stuff_zenith(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
          trace_label=None, log_alt_losers=None, trace_column_names=None, seed=None):
    choices_list = []
    for primary_purpose, trips_segment in trips_merged.groupby('primary_purpose'):
        #print("trip_mode_choice tour_type '%s' (%s trips)" %
        #            (primary_purpose, len(trips_segment.index), ))
        # name index so tracing knows how to slice
        assert trips_segment.index.name == 'trip_id'

        coefficients = simulate.get_segment_coefficients(model_settings, primary_purpose)

        locals_dict = {}
        locals_dict.update(constants)
        locals_dict.update(coefficients)

        segment_trace_label = tracing.extend_trace_label(trace_label, primary_purpose)

        expressions.annotate_preprocessors(
            trips_segment, locals_dict, skims,
            model_settings, segment_trace_label)

        locals_dict.update(skims)

        spec=simulate.eval_coefficients(model_spec, coefficients, estimator)
        nest_spec = simulate.eval_nest_coefficients(nest_spec, coefficients, segment_trace_label)
        choices = simple_simulate_rum_zenith(trips_segment, spec, nest_spec, locals_dict, estimator, 
                                      segment_trace_label, log_alt_losers=log_alt_losers, 
                                      trace_column_names=trace_column_names, seed=seed)

        # for consistency, always return dataframe, whether or not logsums were requested
        if isinstance(choices, pd.Series):
            choices = choices.to_frame('choice')
        choices.rename(columns={'logsum': logsum_column_name,
                                'choice': mode_column_name},
                       inplace=True)
        choices_list.append(choices)

    choices_df = pd.concat(choices_list)
    return choices_df

In [631]:
num_samples = 10

rng_test = default_rng(23423)

print(f"{datetime.now()} Start")
pipeline.open_pipeline(resume_after)
inject.get_injectable('preload_injectables', None)
model_name = "trip_mode_choice"
pipeline._PIPELINE.rng().begin_step(model_name)

step_name = model_name
args = {}
checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

trips = inject.get_table('trips')
tours_merged = inject.get_table('tours_merged')
network_los = inject.get_injectable('network_los')
chunk_size = 0

trace_label = 'trip_mode_choice'
model_settings_file_name = 'trip_mode_choice.yaml'
model_settings = config.read_model_settings(model_settings_file_name)

logsum_column_name = model_settings.get('MODE_CHOICE_LOGSUM_COLUMN_NAME')
mode_column_name = 'trip_mode'
trips_df = trips.to_frame()
#print("Running with %d trips", trips_df.shape[0])
tours_merged = tours_merged.to_frame()
tours_merged = tours_merged[model_settings['TOURS_MERGED_CHOOSER_COLUMNS']]
# - trips_merged - merge trips and tours_merged
trips_merged = pd.merge(
    trips_df,
    tours_merged,
    left_on='tour_id',
    right_index=True,
    how="left")
assert trips_merged.index.equals(trips.index)

# setup skim keys
assert ('trip_period' not in trips_merged)
trips_merged['trip_period'] = network_los.skim_time_period_label(trips_merged.depart)

orig_col = 'origin'
dest_col = 'destination'

constants = {}
constants.update(config.get_model_constants(model_settings))
constants.update({
    'ORIGIN': orig_col,
    'DESTINATION': dest_col
})

skim_dict = network_los.get_default_skim_dict()
odt_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=orig_col, dest_key=dest_col,
                                           dim3_key='trip_period')
dot_skim_stack_wrapper = skim_dict.wrap_3d(orig_key=dest_col, dest_key=orig_col,
                                           dim3_key='trip_period')
od_skim_wrapper = skim_dict.wrap('origin', 'destination')
skims = {
    "odt_skims": odt_skim_stack_wrapper,
    "dot_skims": dot_skim_stack_wrapper,
    "od_skims": od_skim_wrapper,
}
model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
nest_spec = config.get_logit_model_settings(model_settings)
estimator = estimation.manager.begin_estimation('trip_mode_choice')

all_choices_zenith = []
all_choices = []
for i in range(num_samples):
    
    seed = rng_test.integers(0, 100000) #int(9.3 * (i+1)**3)  # why not
    
    if i % 50 == 0:
        print(f"{datetime.now()} iteration {i}")
    choices_df_zenith = stuff_zenith(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
              trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=seed)
    all_choices_zenith.append(choices_df_zenith)

    choices_df = stuff(trips_merged, model_settings, constants, skims, model_spec, nest_spec, estimator, logsum_column_name, mode_column_name, 
          trace_label=trace_label, log_alt_losers=None, trace_column_names=None, seed=seed)
    all_choices.append(choices_df)
    
    t_ = choices_df_zenith.merge(choices_df, left_index=True, right_index=True, suffixes=['_zenith', '_asim'])
    diffs = t_.loc[t_.trip_mode_zenith != t_.trip_mode_asim]
    
    #print(f"seed {seed} leads to {diffs.shape[0]} differences. tripids {diffs.index}")

all_choices_zenith = pd.concat(all_choices_zenith, axis=1)
all_choices = pd.concat(all_choices, axis=1)

probs_nl = gimme_probabilities(trips_merged, model_settings, constants, skims, model_spec, nest_spec,
                               estimator, logsum_column_name, mode_column_name,trace_label=trace_label, 
                               log_alt_losers=None, trace_column_names=None)

finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

print(f"{datetime.now()} End")

In [643]:
print(f"Have {all_choices_zenith.shape[1]} samples")

Have 2038 samples


In [633]:
val_counts_zenith = all_choices_zenith.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts_zenith = val_counts_zenith / all_choices_zenith.shape[1]

val_counts = all_choices.apply(lambda x: x.value_counts(), axis=1).fillna(0)
val_counts = val_counts / all_choices.shape[1]

In [634]:
mode_share_rum_zenith = (val_counts_zenith.sum(axis=0) / val_counts_zenith.shape[0]).to_frame('experiment_zenith')
assert np.allclose(mode_share_rum_zenith.sum(), 1)
full_share_incl_zenith = full_share.merge(mode_share_rum_zenith, left_index=True, right_index=True, how='outer').fillna(0)
with pd.option_context("precision", 3):
    display((100.0 * full_share_incl_zenith).T)

,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,DRIVE_COM,DRIVE_EXP,DRIVE_HVY,DRIVE_LOC,DRIVE_LRF,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_COM,WALK_EXP,WALK_HVY,WALK_LOC,WALK_LRF
analytical,3.152,0.852,0.0,0.0,0.0,0.0,0.0,0.0,0.685,0.0,0.129,0.0,0.182,0.260,1.334,63.708,0.0,0.0,0.455,18.355,10.887
experiment,3.243,0.880,0.0,0.0,0.0,0.0,0.0,0.0,0.772,0.0,0.156,0.0,0.174,0.259,1.434,63.243,0.0,0.0,0.394,18.639,10.807
experiment_zenith,3.150,0.874,0.0,0.0,0.0,0.0,0.0,0.0,0.680,0.0,0.136,0.0,0.154,0.236,1.285,63.767,0.0,0.0,0.515,18.824,10.379


## investigate diverging seed and look at diff in formulation

In [619]:
# TODO: work out if our formulation and formulation based on asim probabilities is equivalent
def compute_nested_utilities_asim(raw_utilities, nest_spec):
    nested_utilities = pd.DataFrame(index=raw_utilities.index)
    for nest in logit.each_nest(nest_spec, post_order=True):
        name = nest.name
        if nest.is_leaf:
            nested_utilities[name] = \
                raw_utilities[name].astype(float) / nest.product_of_coefficients
        else:
            with np.errstate(divide='ignore'):
                nested_utilities[name] = \
                    nest.coefficient * np.log(np.exp(nested_utilities[nest.alternatives]).sum(axis=1))
    return nested_utilities

def compute_nested_utilities_zenith_check(raw_utilities, nest_spec):
    nested_utilities = pd.DataFrame(index=raw_utilities.index)
    for nest in logit.each_nest(nest_spec, post_order=True):
        name = nest.name
        if nest.is_leaf:
            nested_utilities[name] = \
                raw_utilities[name].astype(float)  # scale correction is below
        else:
            with np.errstate(divide='ignore'):
                nested_utilities[name] = \
                    nest.coefficient * np.log(np.exp(nested_utilities[nest.alternatives] / nest.coefficient).sum(axis=1))
                
    # now go over all leaves and correct for scale
    for nest in logit.each_nest(nest_spec):
        name = nest.name
        if nest.is_leaf:
            nested_utilities[name] /= nest.coefficient
    
    return nested_utilities

In [688]:
def simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_d, estimator, tr_label, log_alt_losers, trace_column_names, custom_chooser=None, seed=None, use_zenith=True, raw_utilities=None):
    trace_label = tracing.extend_trace_label(tr_label, 'eval_nl')
    logit.validate_nest_spec(nest_spec, trace_label)
    
    if raw_utilities is None:
        raw_utilities = simulate.eval_utilities(spec, trips_segment, locals_d,
                                       log_alt_losers=log_alt_losers,
                                       trace_label=trace_label, have_trace_targets=False,
                                       estimator=estimator, trace_column_names=trace_column_names)

    if use_zenith:
        utils_df = compute_nested_utilities_zenith_check(raw_utilities, nest_spec)
    else:
        utils_df = compute_nested_utilities_asim(raw_utilities, nest_spec)

    # test
    #return compute_nested_utilities_zenith_check(raw_utilities, nest_spec), compute_nested_utilities_asim(raw_utilities, nest_spec)
        
    nests_ = list(logit.each_nest(nest_spec))
    # group nests by level:
    depth = np.max([x.level for x in nests_])
    nest_levels = {x: [] for x in range(1, depth+1)}
    for n in nests_:
        nest_levels[n.level].append(n.name)
    assert len(nest_levels[1]) == 1
    assert nest_levels[1][0] == 'root'
    # make choices
    choices = make_choice(utils_df, nests_, nest_levels, seed)

    return choices

In [626]:
# t_z, t_a = simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=seed)
# tr_id = 86673661
# display(t_z.loc[t_z.index==tr_id])
# display(t_a.loc[t_a.index==tr_id])

In [625]:
seed = 51006
x_ = simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=seed, use_zenith=True)
y_ = simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=seed, use_zenith=False)
t_ = x_.to_frame('trip_mode_zenith').merge(y_.to_frame('trip_mode_asim'), left_index=True, right_index=True)
diffs = t_.loc[t_.trip_mode_zenith != t_.trip_mode_asim]
print(f"seed {seed} leads to {diffs.shape[0]} differences out of {t_.shape[0]}. tripids {diffs.index}")

seed 51006 leads to 34 differences out of 168. tripids Int64Index([  86673657,   86673661,  106741681,  106741682,  106741685,
             106741686,  106741687,  106741688,  211327433,  211327437,
             444793574,  484173905,  484173909,  535170694,  535620053,
             708171014,  943749470,  943749471, 1060575853, 1091770617,
            1146472489, 1146472493, 1276281769, 1276281773, 1658748793,
            1658748797, 1767013726, 1767186577, 1767186578, 1768237161,
            1768237165, 1768237166, 2463663417, 2463663421],
           dtype='int64', name='trip_id')


In [ ]:
seed 51006 leads to 3 differences. tripids Int64Index([86673661, 535170689, 1060575849], dtype='int64', name='trip_id')


In [753]:
rng_ = default_rng(seed=100)
new_utils = pd.DataFrame(0.1 * -np.log(-np.log(rng_.uniform(0,1,raw_utilities.shape))), columns=raw_utilities.columns)
new_utils.index = raw_utilities.index

In [767]:
nested_exp_utilities_cf = simulate.compute_nested_exp_utilities(new_utils, nest_spec)
nested_probabilities_cf = simulate.compute_nested_probabilities(nested_exp_utilities_cf, nest_spec, trace_label=None)
base_probabilities_cf = simulate.compute_base_probabilities(nested_probabilities_cf, nest_spec, spec)
base_probabilities_cf

,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
trip_id,,,,,,,,,,,,,,,,,,,,,
137248721,0.056642,0.037305,0.042780,0.029576,0.086154,0.026586,0.088313,0.100331,0.028021,0.018516,0.064391,0.020009,0.026399,0.022622,0.023007,0.021754,0.087126,0.021171,0.036475,0.027650,0.135171
137248725,0.101902,0.022876,0.031227,0.069990,0.056913,0.046685,0.088411,0.087785,0.025414,0.038162,0.023988,0.028800,0.039318,0.020160,0.048364,0.023003,0.033959,0.034597,0.037708,0.099523,0.041213
211388201,0.037562,0.070033,0.055951,0.040389,0.020041,0.095974,0.094678,0.094368,0.036014,0.025051,0.034860,0.034953,0.035499,0.028040,0.043650,0.024713,0.031388,0.025931,0.066999,0.052678,0.051228
211388205,0.035480,0.056256,0.043944,0.048085,0.042078,0.062221,0.106819,0.087265,0.035117,0.030399,0.035154,0.026469,0.050646,0.035727,0.024284,0.032905,0.039427,0.041092,0.054714,0.048459,0.063458
806388401,0.054918,0.040488,0.033149,0.054294,0.043477,0.067982,0.152465,0.074152,0.033404,0.028735,0.059269,0.024339,0.036968,0.027760,0.023522,0.060932,0.023445,0.023931,0.044643,0.033773,0.058353
806388405,0.032764,0.057301,0.018437,0.077099,0.046963,0.063521,0.094951,0.089072,0.030826,0.037896,0.035752,0.019878,0.049536,0.079610,0.026922,0.024833,0.024840,0.026695,0.079936,0.040146,0.043022


In [799]:
%%time

num_samples = 10000 # 7.5s per 100

data_zenith = []
data_asim = []
for i in range(num_samples):
    seed = rng_.integers(0, 100000)
    x_ = simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=seed, use_zenith=True, raw_utilities=new_utils)
    y_ = simple_simulate_rum_zenith_check(trips_segment, spec, nest_spec, locals_dict, estimator, tr_label, log_alt_losers, trace_column_names, seed=seed, use_zenith=False, raw_utilities=new_utils)
    data_zenith.append(x_)
    data_asim.append(y_)
    
data_asim = pd.concat(data_asim, axis=1)
data_zenith = pd.concat(data_zenith, axis=1)
# counts_zenith = data_zenith.apply(lambda x: x.value_counts(), axis=1).fillna(0)
# counts_zenith = counts_zenith / data_zenith.shape[1]
# counts_asim = data_asim.apply(lambda x: x.value_counts(), axis=1).fillna(0)
# counts_asim = counts_asim / data_asim.shape[1]

# mode_share_zenith = (counts_zenith.sum(axis=0) / counts_zenith.shape[0]).to_frame('zenith')
# mode_share_asim = (counts_asim.sum(axis=0) / counts_asim.shape[0]).to_frame('asim')
# mode_share_base_prob = (base_probabilities_cf.sum(axis=0) / base_probabilities_cf.shape[0]).to_frame('probs')
# assert np.allclose(mode_share_zenith.sum(), 1)
# assert np.allclose(mode_share_asim.sum(), 1)
# assert np.allclose(mode_share_base_prob.sum(), 1)
# mode_share_comp = mode_share_zenith.join(mode_share_asim, how='outer').join(mode_share_base_prob, how='outer').fillna(0)

CPU times: user 11min 49s, sys: 766 ms, total: 11min 50s
Wall time: 11min 52s


In [796]:
#temp_z = data_zenith.copy()
#temp_a = data_asim.copy()
#data_asim = data_asim.join(temp_a, lsuffix="_o", rsuffix="_n")
#data_zenith = data_zenith.join(temp_z, lsuffix="_o", rsuffix="_n")
# counts_zenith = data_zenith.apply(lambda x: x.value_counts(), axis=1).fillna(0)
# counts_zenith = counts_zenith / data_zenith.shape[1]
# counts_asim = data_asim.apply(lambda x: x.value_counts(), axis=1).fillna(0)
# counts_asim = counts_asim / data_asim.shape[1]

# mode_share_zenith = (counts_zenith.sum(axis=0) / counts_zenith.shape[0]).to_frame('zenith')
# mode_share_asim = (counts_asim.sum(axis=0) / counts_asim.shape[0]).to_frame('asim')
# mode_share_base_prob = (base_probabilities_cf.sum(axis=0) / base_probabilities_cf.shape[0]).to_frame('probs')
# assert np.allclose(mode_share_zenith.sum(), 1)
# assert np.allclose(mode_share_asim.sum(), 1)
# assert np.allclose(mode_share_base_prob.sum(), 1)
# mode_share_comp = mode_share_zenith.join(mode_share_asim, how='outer').join(mode_share_base_prob, how='outer').fillna(0)

In [805]:
# sample of 20000
with pd.option_context("precision", 3):
    display((100.0 * mode_share_comp).T)

,BIKE,DRIVEALONEFREE,DRIVEALONEPAY,DRIVE_COM,DRIVE_EXP,DRIVE_HVY,DRIVE_LOC,DRIVE_LRF,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,TAXI,TNC_SHARED,TNC_SINGLE,WALK,WALK_COM,WALK_EXP,WALK_HVY,WALK_LOC,WALK_LRF
zenith,8.055,5.167,4.631,3.502,3.606,4.266,3.963,3.498,3.895,5.176,4.799,5.851,4.999,6.121,4.668,9.486,4.228,4.125,2.977,3.671,3.317
asim,6.096,6.698,4.977,3.202,3.522,4.334,4.129,3.184,3.224,5.149,6.283,8.296,4.793,6.584,4.532,7.484,4.152,4.273,2.629,3.433,3.023
probs,8.883,5.321,4.738,2.890,3.136,4.003,3.565,3.162,3.758,5.324,4.927,6.049,5.341,6.541,5.037,10.427,3.973,4.224,2.574,3.147,2.979


In [806]:
# TODO: look at probs per trip, i.e. do not sum across trips
trip_id = 137248721

counts_zenith.loc[counts_zenith.index == trip_id].T.merge(
    counts_asim.loc[counts_asim.index == trip_id].T, suffixes=['_z', '_a'], left_index=True, right_index=True, how='outer').merge(
    base_probabilities_cf.loc[base_probabilities_cf.index == trip_id].T, suffixes=['', '_probs'], left_index=True, right_index=True, how='outer').fillna(0)

trip_id,137248721_z,137248721_a,137248721
BIKE,0.09270,0.05435,0.100331
DRIVEALONEFREE,0.05365,0.05005,0.056642
DRIVEALONEPAY,0.03880,0.03155,0.037305
DRIVE_COM,0.02870,0.02470,0.021171
DRIVE_EXP,0.02880,0.02605,0.021754
DRIVE_HVY,0.07815,0.09970,0.087126
DRIVE_LOC,0.02875,0.02700,0.022622
DRIVE_LRF,0.02780,0.02445,0.023007
SHARED2FREE,0.04325,0.02300,0.042780
SHARED2PAY,0.03360,0.01585,0.029576
